In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,081 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,004 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,343 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,269 kB]
Hit:13 https://ppa.launchpadcontent.n

In [ ]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("how to read csv file").getOrCreate()

In [ ]:
# Load in data
df = spark.read.csv(('Resources/heart_clean_df.csv'), header=True, inferSchema=True)

# Show DataFrame
df.show()

+---+----------+---+------+-----------+--------------+----------+--------+--------------+-------+-------+-------------------+-----------------------+---------+-----------------------+--------------+------------+-----------------------+------+------------------+-------------+-------------------------------+-------------------+-------------+-------------+-------------------+-----------------+
|_c0|Patient_ID|Age|   Sex|Cholesterol|Blood_Pressure|Heart_Rate|Diabetes|Family_History|Smoking|Obesity|Alcohol_Consumption|Exercise_Hours_Per_Week|     Diet|Previous_Heart_Problems|Medication_Use|Stress_Level|Sedentary_Hours_Per_Day|Income|               BMI|Triglycerides|Physical_Activity_Days_Per_Week|Sleep_Hours_Per_Day|      Country|    Continent|         Hemisphere|Heart_Attack_Risk|
+---+----------+---+------+-----------+--------------+----------+--------+--------------+-------+-------+-------------------+-----------------------+---------+-----------------------+--------------+------------+-

In [5]:
# Create our temporary view
df.createOrReplaceTempView('heart')

In [6]:
# Demographic analysis - Total Number of date with each gender
spark.sql("""
  SELECT Sex,
  Count (Sex) as Number_of_Patients
  FROM heart
  GROUP BY Sex
  """).show()

+------+------------------+
|   Sex|Number_of_Patients|
+------+------------------+
|Female|              2652|
|  Male|              6111|
+------+------------------+



In [7]:
# Demographic analysis - Nunber of each gender with heart attack risk
spark.sql("""
  SELECT Sex,
  Count (Sex) as Number_of_Patients
  FROM heart
  WHERE Heart_Attack_Risk = 1
  GROUP BY Sex
  """).show()

+------+------------------+
|   Sex|Number_of_Patients|
+------+------------------+
|Female|               944|
|  Male|              2195|
+------+------------------+



In [8]:
# Demographic analysis - Age group > 50, difference sex with pervious history and potential heart attack
spark.sql("""
  SELECT Sex,
  Count (Sex) as Older_Than_50
  FROM heart
  WHERE Age > 50 and Previous_Heart_Problems =1 and Heart_Attack_Risk = 1
  GROUP BY Sex
  """).show()

+------+-------------+
|   Sex|Older_Than_50|
+------+-------------+
|Female|          237|
|  Male|          597|
+------+-------------+



In [9]:
# Demographic analysis - Age group > 50, difference sex with potential heart attack
spark.sql("""
  SELECT Sex,
  Count (Sex) as Older_Than_50
  FROM heart
  WHERE Age > 50 and Heart_Attack_Risk = 1
  GROUP BY Sex
  """).show()

+------+-------------+
|   Sex|Older_Than_50|
+------+-------------+
|Female|          489|
|  Male|         1189|
+------+-------------+



In [10]:
# Demographic analysis - Age group < 30, difference sex with pervious history and potential heart attack
spark.sql("""
  SELECT Sex,
  COUNT (Sex) as Yonger_Then_30
  FROM heart
  WHERE Age < 30 and Previous_Heart_Problems =1 and Heart_Attack_Risk = 1
  GROUP BY Sex
  """).show()

+------+--------------+
|   Sex|Yonger_Then_30|
+------+--------------+
|Female|            94|
|  Male|           171|
+------+--------------+



In [11]:
# Demographic analysis - Age group < 30, difference sex with potential heart attack
spark.sql("""
  SELECT Sex,
  COUNT (Sex) as Yonger_Then_30
  FROM heart
  WHERE Age < 30 and Heart_Attack_Risk = 1
  GROUP BY Sex
  """).show()

+------+--------------+
|   Sex|Yonger_Then_30|
+------+--------------+
|Female|           169|
|  Male|           372|
+------+--------------+



In [12]:
# Demographic analysis - Avg Income
spark.sql("""
  SELECT Heart_Attack_Risk,
  AVG(Income) as Avg_Income
  FROM heart
  GROUP BY Heart_Attack_Risk
  """).show()

+-----------------+-----------------+
|Heart_Attack_Risk|       Avg_Income|
+-----------------+-----------------+
|                1| 159301.479133482|
|                0|157683.6628733997|
+-----------------+-----------------+



In [13]:
# Demographic analysis - Patient has potential heart attack in different Country
spark.sql("""
  SELECT Country,
  COUNT(Country) as Number_of_Patients
  FROM heart
  GROUP BY Country
  """).show()

+--------------+------------------+
|       Country|Number_of_Patients|
+--------------+------------------+
|       Germany|               477|
|        France|               446|
|     Argentina|               471|
| United States|               420|
|         China|               436|
|         India|               412|
|       Nigeria|               448|
|         Italy|               431|
|         Spain|               430|
|      Thailand|               428|
|   South Korea|               409|
|        Canada|               440|
|        Brazil|               462|
|         Japan|               433|
|   New Zealand|               435|
|     Australia|               449|
|  South Africa|               425|
|      Colombia|               429|
|United Kingdom|               457|
|       Vietnam|               425|
+--------------+------------------+



In [14]:
# Lifestyle analysis - relationship with Diabetes, Family_History, Smoking, Obesity, Alcohol_Consumption
spark.sql("""
  SELECT Heart_Attack_Risk,
  COUNT(Heart_Attack_Risk) as Number_of_Patients
  FROM heart
  WHERE Diabetes =1 and Family_History =1 and Smoking =1 and Obesity =1 and Alcohol_Consumption =1
  GROUP BY Heart_Attack_Risk
  """).show()

+-----------------+------------------+
|Heart_Attack_Risk|Number_of_Patients|
+-----------------+------------------+
|                1|               284|
|                0|               494|
+-----------------+------------------+



In [15]:
# Lifestyle analysis - Average Sleeping per day
spark.sql("""
  SELECT Heart_Attack_Risk,
  AVG(Sleep_Hours_Per_Day) as Avg_Sleeping_Time_Per_Day
  FROM heart
  GROUP BY Heart_Attack_Risk
  """).show()

+-----------------+-------------------------+
|Heart_Attack_Risk|Avg_Sleeping_Time_Per_Day|
+-----------------+-------------------------+
|                1|        6.974195603695445|
|                0|        7.051031294452347|
+-----------------+-------------------------+



In [17]:
# Lifestyle analysis - Diet with exercise with potential heart attack
spark.sql("""
  SELECT Diet,
  AVG(Exercise_Hours_Per_Week) as Avg_Exercise_Hours_Per_Week
  FROM heart
  WHERE Heart_Attack_Risk = 1
  GROUP BY Diet
  """).show()

+---------+---------------------------+
|     Diet|Avg_Exercise_Hours_Per_Week|
+---------+---------------------------+
|Unhealthy|         10.179744647675276|
|  Average|          9.987016034593525|
|  Healthy|         10.132370137728234|
+---------+---------------------------+



In [18]:
# Lifestyle analysis - Diet with exercise without potential heart attack
spark.sql("""
  SELECT Diet,
  AVG(Exercise_Hours_Per_Week) as Avg_Exercise_Hours_Per_Week
  FROM heart
  WHERE Heart_Attack_Risk = 0
  GROUP BY Diet
  """).show()

+---------+---------------------------+
|     Diet|Avg_Exercise_Hours_Per_Week|
+---------+---------------------------+
|Unhealthy|          9.884804984010403|
|  Average|          9.934329292998868|
|  Healthy|          10.07845718923255|
+---------+---------------------------+

